In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cafa-5-protein-function-prediction/sample_submission.tsv
/kaggle/input/cafa-5-protein-function-prediction/IA.txt
/kaggle/input/cafa-5-protein-function-prediction/Test (Targets)/testsuperset.fasta
/kaggle/input/cafa-5-protein-function-prediction/Test (Targets)/testsuperset-taxon-list.tsv
/kaggle/input/cafa-5-protein-function-prediction/Train/train_terms.tsv
/kaggle/input/cafa-5-protein-function-prediction/Train/train_sequences.fasta
/kaggle/input/cafa-5-protein-function-prediction/Train/train_taxonomy.tsv
/kaggle/input/cafa-5-protein-function-prediction/Train/go-basic.obo
/kaggle/input/protbert-embeddings-for-cafa5/train_ids.npy
/kaggle/input/protbert-embeddings-for-cafa5/train_embeddings.npy
/kaggle/input/protbert-embeddings-for-cafa5/test_ids.npy
/kaggle/input/protbert-embeddings-for-cafa5/test_embeddings.npy


In [2]:
#import libraiares:
import tensorflow as tf
import pandas as pd
import numpy as np

# we will use bio python to read the sequences
from Bio import SeqIO

import seaborn as sns
import matplotlib.pyplot as plt


from sklearn.impute import KNNImputer, SimpleImputer
#import statsmodels.api as sm

from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
#from sklearn.ensemble import RandomForestRegressor
from scipy.stats import spearmanr
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler
import re
from scipy.stats import randint

import plotly.express as px
from transformers import BertModel, BertTokenizer
import torch

# Required for progressbar widget
import progressbar
import time
t0start = time.time() 
from tqdm import tqdm

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

# Problem definition:
The goal of this competition is to predict the function of a set of proteins.
The accurate assignment of biological function to the protein is key to understanding life at the molecular level. However, assigning function to any specific protein can be made difficult due to the multiple functions many proteins have, along with their ability to interact with multiple partners.
We are provided with a set of protein sequences on which the participants are asked to predict Gene Ontology (GO) terms in each of the three subontologies: Molecular Function (MF), Biological Process (BP), and Cellular Component (CC). This set of sequences is referred to as test superset.
# Data:
1. train_sequences.fasta - amino acid sequences for proteins in training set
2. testsuperset.fasta - amino acid sequences for proteins on which the predictions should be made
3. train_terms.tsv - the training set of proteins and corresponding annotated GO terms
4. train_taxonomy.tsv - taxon ID for proteins in training set
5. go-basic.obo - ontology graph structure
6. testsuperset-taxon-list.tsv - taxon ID for proteins in test superset (Note: you may need to use encoding="ISO-8859-1" to read this file in pandas)
7. IA.txt - Information Accretion for each term. This is used to weight precision and recall (see Evaluation)
8. sample_submission.csv - a sample submission file in the correct format

In [3]:
# path to the train and test fasta files
train_fasta = '/kaggle/input/cafa-5-protein-function-prediction/Train/train_sequences.fasta'
test_fasta = '/kaggle/input/cafa-5-protein-function-prediction/Test (Targets)/testsuperset.fasta'

In [4]:
# read train fasta files
train_sequences = SeqIO.parse(train_fasta, 'fasta')

In [5]:
sequences_dict = SeqIO.to_dict(train_sequences)

In [6]:
# take a look at the first three instances:
first_three_keys = list(sequences_dict.keys())[:3]
first_three_instances = {key: sequences_dict[key] for key in first_three_keys}

# Printing the first three instances
for seq_id, seq_record in first_three_instances.items():
    print("ID:", seq_id)
    print("Sequence:", seq_record.seq)
    print('Description:', seq_record.description)
    print()

ID: P20536
Sequence: MNSVTVSHAPYTITYHDDWEPVMSQLVEFYNEVASWLLRDETSPIPDKFFIQLKQPLRNKRVCVCGIDPYPKDGTGVPFESPNFTKKSIKEIASSISRLTGVIDYKGYNLNIIDGVIPWNYYLSCKLGETKSHAIYWDKISKLLLQHITKHVSVLYCLGKTDFSNIRAKLESPVTTIVGYHPAARDRQFEKDRSFEIINVLLELDNKVPINWAQGFIY
Description: P20536 sp|P20536|UNG_VACCC Uracil-DNA glycosylase OS=Vaccinia virus (strain Copenhagen) OX=10249 GN=UNG PE=1 SV=1

ID: O73864
Sequence: MTEYRNFLLLFITSLSVIYPCTGISWLGLTINGSSVGWNQTHHCKLLDGLVPDQQQLCKRNLELMHSIVRAARLTKSACTSSFSDMRWNWSSIESAPHFTPDLAKGTREAAFVVSLAAAVVSHAIARACASGDLPSCSCAAMPSEQAAPDFRWGGCGDNLRYYGLQMGSAFSDAPMRNRRSGPQDFRLMQLHNNAVGRQVLMDSLEMKCKCHGVSGSCSVKTCWKGLQDISTISADLKSKYLSATKVIPRQIGTRRQLVPREMEVRPVGENELVYLVSSPDYCTQNAKQGSLGTTDRQCNKTASGSESCGLMCCGRGYNAYTEVLVERCQCKYHWCCYVSCKTCKRTVERYVSK
Description: O73864 sp|O73864|WNT11_DANRE Protein Wnt-11 OS=Danio rerio OX=7955 GN=wnt11 PE=2 SV=1

ID: O95231
Sequence: MRLSSSPPRGPQQLSSFGSVDWLSQSSCSGPTHTPRPADFSLGSLPGPGQTSGAREPPQAVSIKEAAGSSNLPAPERTMAGLSKEPNTLRAPRVRTAFTMEQVRTLEGVFQHHQYLSPLERKRLAREMQLSEVQI

In [7]:
#create a dataframe of the above fasta files dictionary:
data = []
for seq_id, seq_record in sequences_dict.items():
    sequence = str(seq_record.seq)
    seq_des = str(seq_record.description)
    data.append({"ID": seq_id, "Sequence": sequence, "description": seq_des})

In [8]:
df_sequences = pd.DataFrame(data)

In [9]:
df_sequences.head()

,ID,Sequence,description
0,P20536,MNSVTVSHAPYTITYHDDWEPVMSQLVEFYNEVASWLLRDETSPIP...,P20536 sp|P20536|UNG_VACCC Uracil-DNA glycosyl...
1,O73864,MTEYRNFLLLFITSLSVIYPCTGISWLGLTINGSSVGWNQTHHCKL...,O73864 sp|O73864|WNT11_DANRE Protein Wnt-11 OS...
2,O95231,MRLSSSPPRGPQQLSSFGSVDWLSQSSCSGPTHTPRPADFSLGSLP...,O95231 sp|O95231|VENTX_HUMAN Homeobox protein ...
3,A0A0B4J1F4,MGGEAGADGPRGRVKSLGLVFEDESKGCYSSGETVAGHVLLEAAEP...,A0A0B4J1F4 sp|A0A0B4J1F4|ARRD4_MOUSE Arrestin ...
4,P54366,MVETNSPPAGYTLKRSPSDLGEQQQPPRQISRSPGNTAAYHLTTAM...,P54366 sp|P54366|GSC_DROME Homeobox protein go...


In [10]:
#create a column with length of sequences:
df_sequences['sequence_len'] = df_sequences['Sequence'].apply(len)

In [11]:
# read test fasta files
test_sequences = SeqIO.parse(test_fasta, 'fasta')

In [12]:
# Remove duplicates based on ID
test_sequences = {rec.id: rec for rec in test_sequences}.values()

# Convert the sequences to a dictionary
sequences_dict_test = SeqIO.to_dict(test_sequences)


In [13]:
#create a dataframe of the above fasta files dictionary:
data_test = []
for seq_id, seq_record in sequences_dict_test.items():
    sequence = str(seq_record.seq)
    seq_des = str(seq_record.description)
    data_test.append({"ID": seq_id, "Sequence": sequence, "description": seq_des})

df_sequences_test = pd.DataFrame(data_test)

In [14]:
#create a column with length of sequences:
df_sequences_test['sequence_len'] = df_sequences_test['Sequence'].apply(len)

In [15]:
df_sequences_test.head()

,ID,Sequence,description,sequence_len
0,Q9CQV8,MTMDKSELVQKAKLAEQAERYDDMAAAMKAVTEQGHELSNEERNLL...,Q9CQV8\t10090,246
1,P62259,MDDREDLVYQAKLAEQAERYDEMVESMKKVAGMDVELTVEERNLLS...,P62259\t10090,255
2,P68510,MGDREQLLQRARLAEQAERYDDMASAMKAVTELNEPLSNEDRNLLS...,P68510\t10090,246
3,P61982,MVDREQLVQKARLAEQAERYDDMAAAMKNVTELNEPLSNEERNLLS...,P61982\t10090,247
4,O70456,MERASLIQKAKLAEQAERYEDMAAFMKSAVEKGEELSCEERNLLSV...,O70456\t10090,248


In [16]:
#Load the terms /label data:
train_terms = pd.read_csv("/kaggle/input/cafa-5-protein-function-prediction/Train/train_terms.tsv",sep="\t")
print(train_terms.shape)

(5363863, 3)


In [17]:
#Load the taxanomy Dataset
train_tax = pd.read_csv("/kaggle/input/cafa-5-protein-function-prediction/Train/train_taxonomy.tsv",sep="\t")
print(train_tax.shape)

(142246, 2)


 
 # A. Data preprocessing:
1. check datasets for null values: datasets are free of null values.

2. Create labels for the model:

First we will extract all the needed labels(GO term ID) from train_terms.tsv file. There are more than 40,000 labels. In order to simplify our model, we will choose the most frequent 1500 GO term IDs as labels. 

Then we will fetch the train_terms data for the relevant labels only. and then we will create the The final labels dataframe (label_df),  composed of 1500 columns and 142246 entries. 

3. Create embeddings for the protein sequences. We will be using pretrained embeddings form protBERT to


## 1. check data set for any missing values

In [18]:
#check sequences for null values:
df_sequences.isna().sum()

ID              0
Sequence        0
description     0
sequence_len    0
dtype: int64

In [19]:
#check sequences for null values:
df_sequences_test.isna().sum()

ID              0
Sequence        0
description     0
sequence_len    0
dtype: int64

In [20]:
#check data for null values:
train_terms.isna().sum()

EntryID    0
term       0
aspect     0
dtype: int64

In [21]:
df_sequences.describe()

,sequence_len
count,142246.000000
mean,553.636679
std,641.728770
min,3.000000
25%,248.000000
50%,411.000000
75%,654.000000
max,35375.000000


## 2. get labels as label_df
First we will extract all the needed labels(GO term ID) from train_terms.tsv file.

There are more than 40,000 labels. In order to simplify our model, we will choose the most frequent 1500 GO term IDs as labels.|

In [41]:
# We will now save the first 1500 most frequent GO term Ids into a list.
# Set the limit for label
num_of_labels = 1500

# Take value counts in descending order and fetch first 1500 `GO term ID` as labels
labels = train_terms['term'].value_counts().index[:num_of_labels].tolist()

In [42]:
# Fetch the train_terms data for the relevant labels only
train_terms_updated = train_terms.loc[train_terms['term'].isin(labels)]
train_terms_updated.shape

(4420737, 3)

In [43]:
train_terms_updated

,EntryID,term,aspect
0,A0A009IHW8,GO:0008152,BPO
1,A0A009IHW8,GO:0034655,BPO
3,A0A009IHW8,GO:0044270,BPO
4,A0A009IHW8,GO:0006753,BPO
6,A0A009IHW8,GO:0044237,BPO
...,...,...,...
5363857,X5L565,GO:0003674,MFO
5363859,X5L565,GO:0016491,MFO
5363860,X5M5N0,GO:0005515,MFO
5363861,X5M5N0,GO:0005488,MFO


In [24]:
train_protein_ids = np.load('/kaggle/input/protbert-embeddings-for-cafa5/train_ids.npy')
print(train_protein_ids.shape)

(142246,)


In [ ]:
# Setup progressbar settings.
# This is strictly for aesthetic.
bar = progressbar.ProgressBar(maxval=num_of_labels, \
    widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])

# Create an empty dataframe of required size for storing the labels,
# i.e, train_size x num_of_labels (1422a46 x 1500)
train_size = train_protein_ids.shape[0] # len(X)
train_labels = np.zeros((train_size ,num_of_labels))

# Convert from numpy to pandas series for better handling
series_train_protein_ids = pd.Series(train_protein_ids)

# Loop through each label
for i in range(num_of_labels):
    # For each label, fetch the corresponding train_terms data
    n_train_terms = train_terms_updated[train_terms_updated['term'] ==  labels[i]]
    
    # Fetch all the unique EntryId aka proteins related to the current label(GO term ID)
    label_related_proteins = n_train_terms['EntryID'].unique()
    
    # In the series_train_protein_ids pandas series, if a protein is related
    # to the current label, then mark it as 1, else 0.
    # Replace the ith column of train_Y with with that pandas series.
    train_labels[:,i] =  series_train_protein_ids.isin(label_related_proteins).astype(float)
    
    # Progress bar percentage increase
    bar.update(i+1)

# Notify the end of progress bar 
bar.finish()

# Convert train_Y numpy into pandas dataframe
labels_df = pd.DataFrame(data = train_labels, columns = labels)
print(labels_df.shape)

The final labels dataframe (label_df) is composed of 1500 columns and 142246 entries. We can see all 1500 dimensions(results will be truncated since the number of columns is big) of our dataset by printing out the first 5 entries using the following code:

In [ ]:
labels_df.head()

Label을 aspect로 바꾸고 BPO, MFO, CCO 로 onhit라벨링 해야하나?

In [36]:
# Take value counts in descending order and fetch first 1500 `GO term ID` as labels
labels = train_terms['aspect'].value_counts()
labels

BPO    3497732
CCO    1196017
MFO     670114
Name: aspect, dtype: int64

In [38]:
labels = ['BPO', 'CCO', 'MFO']
labels

['BPO', 'CCO', 'MFO']

In [39]:
# Fetch the train_terms data for the relevant labels only
train_terms_updated = train_terms.loc[train_terms['aspect'].isin(labels)]
train_terms_updated.shape

(5363863, 3)

In [40]:
train_terms_updated

,EntryID,term,aspect
0,A0A009IHW8,GO:0008152,BPO
1,A0A009IHW8,GO:0034655,BPO
2,A0A009IHW8,GO:0072523,BPO
3,A0A009IHW8,GO:0044270,BPO
4,A0A009IHW8,GO:0006753,BPO
...,...,...,...
5363858,X5L565,GO:0050649,MFO
5363859,X5L565,GO:0016491,MFO
5363860,X5M5N0,GO:0005515,MFO
5363861,X5M5N0,GO:0005488,MFO


In [47]:
train_protein_ids = np.load('/kaggle/input/protbert-embeddings-for-cafa5/train_ids.npy')
train_protein_ids

array(['P20536', 'O73864', 'O95231', ..., 'Q5RGB0', 'A0A2R8QMZ5',
       'A0A8I6GHU0'], dtype='<U10')

In [49]:
# Setup progressbar settings.
# This is strictly for aesthetic.
bar = progressbar.ProgressBar(maxval=num_of_labels, \
    widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])

# Create an empty dataframe of required size for storing the labels,
# i.e, train_size x num_of_labels (1422a46 x 3)
train_size = train_protein_ids.shape[0] # len(X)
train_labels = np.zeros((train_size ,num_of_labels))

# Convert from numpy to pandas series for better handling
series_train_protein_ids = pd.Series(train_protein_ids)

# Loop through each label
for i in range(num_of_labels):
    # For each label, fetch the corresponding train_terms data
    n_train_terms = train_terms_updated[train_terms_updated['aspect'] ==  labels[i]]
    
    # Fetch all the unique EntryId aka proteins related to the current label(GO term ID)
    label_related_proteins = n_train_terms['EntryID'].unique()
    
    # In the series_train_protein_ids pandas series, if a protein is related
    # to the current label, then mark it as 1, else 0.
    # Replace the ith column of train_Y with with that pandas series.
    train_labels[:,i] =  series_train_protein_ids.isin(label_related_proteins).astype(float)
    
    # Progress bar percentage increase
    bar.update(i+1)

# Notify the end of progress bar 
bar.finish()

# Convert train_Y numpy into pandas dataframe
labels_df = pd.DataFrame(data = train_labels, columns = labels)
print(labels_df.shape)

[====================                                                    ]  29%

KeyboardInterrupt: 

## Get the sequence embeddings using protBERT

### What is ProtBert model?
Pretrained model on protein sequences using a masked language modeling (MLM) objective. ProtBert is based on Bert model which pretrained on a large corpus of protein sequences in a self-supervised fashion. This means it was pretrained on the raw protein sequences only, with no humans labelling them in any way (which is why it can use lots of publicly available data) with an automatic process to generate inputs and labels from those protein sequences.

One important difference between our Bert model and the original Bert version is the way of dealing with sequences as separate documents. This means the Next sentence prediction is not used, as each sequence is treated as a complete document. The masking follows the original Bert training with randomly masks 15% of the amino acids in the input.

At the end, the feature extracted from this model revealed that the LM-embeddings from unlabeled data (only protein sequences) captured important biophysical properties governing protein shape. This implied learning some of the grammar of the language of life realized in protein sequences.

ref: https://huggingface.co/Rostlab/prot_bert

The precalculated protein embeddings can be found here: https://www.kaggle.com/datasets/henriupton/protbert-embeddings-for-cafa5

In [ ]:
%%time
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tokenizer = BertTokenizer.from_pretrained("Rostlab/prot_bert", do_lower_case=False )
model = BertModel.from_pretrained("Rostlab/prot_bert").to(device)

Lets take a moment to see how amino acids are tokenized by prot-BERT model

In [ ]:
#get the token embedding for amino acids
token_embedding = {token: model.get_input_embeddings()(torch.tensor(idx)) for token, idx in tokenizer.get_vocab().items()}

In [ ]:
print(len(token_embedding))

In [ ]:
#print embedding of aminoacid == ALANINE(A)
print(token_embedding['A'])

In [ ]:
print(token_embedding)

We will now load the pre calculated protein embeddings created by HENRI UPTON using the Rost Lab's protBERT protein language model.

The protein embeddings to be used for training are recorded in train_embeddings.npy and the corresponding protein ids are available in train_ids.npy.

First, we will load the protein ids of the protein embeddings in the train dataset contained in train_ids.npy into a numpy array.

The train_protein_ids array consists of 142246 protein_ids. Let us print out the first 5 entries using the following code:

In [ ]:
train_protein_ids[:5]

After loading the files as numpy arrays, we will convert them into Pandas dataframe.

Each protein embedding is a vector of length 1024. We create the resulting dataframe such that there are 1024 columns to represent the values in each of the 1024 places in the vector.

In [ ]:
#get pretrained embeddings:

import numpy as np
# Load the .npy file
data_embed = np.load('/kaggle/input/protbert-embeddings-for-cafa5/train_embeddings.npy')

In [ ]:
#get temp df of the embeddings:
train_df = pd.DataFrame(data_embed)
#del embeddings_list
train_df .columns = ['Feature_' + str(x) for x in train_df.columns]
train_df 

In [ ]:
test_embed = np.load('/kaggle/input/protbert-embeddings-for-cafa5/test_embeddings.npy')
#get temp df for test data embeddings
test_df = pd.DataFrame(test_embed)
#del test_embeddings_list
test_df.columns = ['Feature_' + str(x) for x in test_df.columns]
test_df

# Model:
Next, we will use Tensorflow to train a Deep Neural Network with the protein embeddings.

## 1. define X and y

## 2. Prepare train-test split

In [ ]:
# train and validation set
X = train_df.to_numpy()
y = labels_df.to_numpy()

In [ ]:
print(X.shape, y.shape)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

print(X_train.shape, X_val.shape)
print(y_train.shape, y_val.shape)

## 3. ANN model

In [ ]:
INPUT_SHAPE = [X_train.shape[1]]
BATCH_SIZE = 5120

model = tf.keras.Sequential([
    tf.keras.layers.BatchNormalization(input_shape=INPUT_SHAPE),    
    tf.keras.layers.Dense(units=512, activation='relu'),
    tf.keras.layers.Dense(units=512, activation='relu'),
    tf.keras.layers.Dense(units=512, activation='relu'),
    tf.keras.layers.Dense(units=num_of_labels,activation='sigmoid')
])


# Compile model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['binary_accuracy', tf.keras.metrics.AUC()],
)

In [ ]:
# callbacks
earlystopping_cb = tf.keras.callbacks.EarlyStopping(monitor="loss", patience=10)
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("/kaggle/working/best_model.pro5", save_best_only=True)

In [ ]:
# train new model
history = model.fit(X_train, y_train, epochs=30,
                       validation_data=(X_val, y_val),
                       callbacks=[earlystopping_cb, checkpoint_cb])

# Plot the model's loss and accuracy for each epoch
A validation accuracy of  0.90


The history.history variable contains the information of each epoch.



In [ ]:
history_df = pd.DataFrame(history.history)
history_df.loc[:, ['loss']].plot(title="Cross-entropy")
history_df.loc[:, ['binary_accuracy']].plot(title="Accuracy")

# Submission

Load features for submission

In [ ]:
%%time
# fn = '/kaggle/input/protein-embeddings-1/reduced_embeddings_file.npy'
# fn = '/kaggle/input/protein-embeddings-1/embed_protbert_train_clip_1200_first_70000_prot.csv'
# fn = '/kaggle/input/t5embeds/train_embeds.npy'
fn = '/kaggle/input/protbert-embeddings-for-cafa5/test_embeddings.npy'
print(fn)
X_submit = np.load(fn)
print(X_submit.shape)
# X_submit

Calculate prediction for submission

In [ ]:
%%time
Y_submit =  model.predict(X_submit)
print(Y_submit.shape)

In [ ]:
%%time 
df_finalSubmission = pd.DataFrame(columns = ['Protein Id', 'GO Term Id','Prediction'])

Load protein ids for the submission

In [ ]:
%%time
fn = '/kaggle/input/protbert-embeddings-for-cafa5/test_ids.npy'
vec_test_protein_ids = np.load(fn)
print(vec_test_protein_ids.shape)
vec_test_protein_ids

"Melt" protein ids

In [ ]:
%%time 
l = []
for k in list(vec_test_protein_ids):
    l += [ k] * Y_submit.shape[1]
print(len(l), l[:20])    

df_finalSubmission['Protein Id'] = l

"Melt" Labels (Gene ontology terms )

In [ ]:
df_finalSubmission['GO Term Id'] = labels * Y_submit.shape[0]
# df_finalSubmission.head(3)

Assign predictions

In [ ]:
df_finalSubmission['Prediction'] = Y_submit.ravel()

Save

In [ ]:
%%time
df_finalSubmission.to_csv("submission.tsv",header=False, index=False, sep="\t")

In [ ]:
df_finalSubmission.head()